In [11]:
import folium
from folium.plugins import MarkerCluster
import os
import json
import branca.colormap as cm

# database imports
from sqlalchemy import create_engine, text, Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base
from geoalchemy2 import Geometry, WKTElement

# data models
from model import Base, FeatureSet, Feature, Region, Point, Colormap

In [12]:
# connect to PostGIS
password = "rescuemate" # TODO: change this later to something proper

DATABASE_URL = "postgresql+psycopg2://postgres:" + password + "@localhost:5432/postgres"
engine = create_engine(DATABASE_URL, echo=True)

# Start a session
Session = sessionmaker(bind=engine)
session = Session()

# activate postGIS if its not already enabled
session.execute(text("CREATE EXTENSION IF NOT EXISTS postgis"))

# add the data models to the database
Base.metadata.create_all(engine)

2023-10-21 14:00:03,969 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-10-21 14:00:03,970 INFO sqlalchemy.engine.Engine [raw sql] {}


2023-10-21 14:00:03,972 INFO sqlalchemy.engine.Engine select current_schema()
2023-10-21 14:00:03,973 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-21 14:00:03,977 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-10-21 14:00:03,978 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-21 14:00:03,980 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-21 14:00:03,982 INFO sqlalchemy.engine.Engine CREATE EXTENSION IF NOT EXISTS postgis
2023-10-21 14:00:03,984 INFO sqlalchemy.engine.Engine [generated in 0.00145s] {}
2023-10-21 14:00:04,012 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-21 14:00:04,018 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catal

In [18]:
# drop all tables
Base.metadata.drop_all(engine)

2023-10-21 14:03:12,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-21 14:03:12,351 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2023-10-21 14:03:12,351 INFO sqlalchemy.engine.Engine [cached since 188.3s ago] {'table_name': 'feature_sets', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2023-10-21 14:03:12,355 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catal

In [8]:
# if necessary, rollback
session.rollback()

C:\Users\David Rath\AppData\Local\Temp\ipykernel_13504\1801316652.py:2: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  session.rollback()


In [16]:
# Insert some dummy data into the database
# Create a new test FeatureSet
feature_set = FeatureSet(
    name="Sample FeatureSet",
    color="#FF5733",  # Sample color
    fill_color="#33FF57",  # Sample fill color
    popup_properties={
        "Name": "name",
        "Description": "description"
    }
)

# Create a new test Point within the FeatureSet
new_point = Point(
    properties={
        "name": "Sample Point",
        "description": "This is a test point."
    },
    geometry=WKTElement("POINT(2 2)"),
    icon_prefix="fa",   # FontAwesome
    icon_name="circle"  # circle icon
)

# Create a new test Region within the FeatureSet
new_region = Region(
    properties={
        "name": "Sample Region",
        "description": "This is a test region."
    },
    geometry=WKTElement("POLYGON((0 0, 1 0, 1 1, 0 1, 0 0))"),
    line_weight=2.0,
    geometry_type="POLYGON"
)

# Link the Point and Region to the FeatureSet
feature_set.features = [new_point, new_region]

# Add and commit the new FeatureSet, Point, and Region to the database
session.add(feature_set)
session.commit()

2023-10-21 14:03:02,734 INFO sqlalchemy.engine.Engine INSERT INTO feature_sets (name, color, fill_color, colormap_id, popup_properties) VALUES (%(name)s, %(color)s, %(fill_color)s, %(colormap_id)s, %(popup_properties)s) RETURNING feature_sets.id
2023-10-21 14:03:02,735 INFO sqlalchemy.engine.Engine [cached since 173.1s ago] {'name': 'Sample FeatureSet', 'color': '#FF5733', 'fill_color': '#33FF57', 'colormap_id': None, 'popup_properties': '{"Name": "name", "Description": "description"}'}
2023-10-21 14:03:02,737 INFO sqlalchemy.engine.Engine INSERT INTO features (feature_set_id, properties, type) SELECT p0::INTEGER, p1::JSON, p2::VARCHAR FROM (VALUES (%(feature_set_id__0)s, %(properties__0)s, %(type__0)s, 0), (%(feature_set_id__1)s, %(properties__1)s, %(type__1)s, 1)) AS imp_sen(p0, p1, p2, sen_counter) ORDER BY sen_counter RETURNING features.id, features.id AS id__1
2023-10-21 14:03:02,738 INFO sqlalchemy.engine.Engine [cached since 173.1s ago (insertmanyvalues) 1/1 (ordered)] {'type__0

In [17]:
# query all values in all FeatureSets
feature_sets = session.query(FeatureSet).all()
for feature_set in feature_sets:
    print(feature_set.id, feature_set.name, feature_set.color, feature_set.fill_color)
    for feature in feature_set.features:
        print(feature.id, feature.properties["name"], feature.geometry)

2023-10-21 14:03:04,727 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-21 14:03:04,727 INFO sqlalchemy.engine.Engine SELECT feature_sets.id AS feature_sets_id, feature_sets.name AS feature_sets_name, feature_sets.color AS feature_sets_color, feature_sets.fill_color AS feature_sets_fill_color, feature_sets.colormap_id AS feature_sets_colormap_id, feature_sets.popup_properties AS feature_sets_popup_properties 
FROM feature_sets
2023-10-21 14:03:04,728 INFO sqlalchemy.engine.Engine [cached since 139.2s ago] {}
1 Sample FeatureSet #FF5733 #33FF57
2023-10-21 14:03:04,731 INFO sqlalchemy.engine.Engine SELECT features.id AS features_id, features.feature_set_id AS features_feature_set_id, features.properties AS features_properties, features.type AS features_type 
FROM features 
WHERE %(param_1)s = features.feature_set_id
2023-10-21 14:03:04,732 INFO sqlalchemy.engine.Engine [cached since 139.2s ago] {'param_1': 1}
2023-10-21 14:03:04,735 INFO sqlalchemy.engine.Engine SELECT points.icon

# Handling data in PostGIS

### Inserting
**Single Record**:
```python
location1 = Location(name="Point A", coordinate="POINT(0 0)")
session.add(location1)
session.commit()
```

**Multiple Records**:
```python
locations_list = [
    Location(name="Point B", coordinate="POINT(1 1)"),
    Location(name="Point C", coordinate="POINT(2 2)")
]
session.add_all(locations_list)
session.commit()
```

### Querying
**Query all**:
```python
locations = session.query(Location).all()
for location in locations:
    print(location.name, location.coordinate)
```

**Query with filter**:
```python
point_b = session.query(Location).filter_by(name="Point B").first()
print(point_b.id, point_b.name, point_b.geom)
```

### Updating
```python
point_b.name = "Point B Updated"
session.commit()
```

### Deleting
**Single record**:
```python
session.delete(point_b)
session.commit()
```

**All locations**:
```python
session.query(Location).delete()
session.commit()
```

In [2]:
# gather all geojson files
# geojson files are located in their respective folders under data
# files are then saved in a dictionary with the folder name as key

# TODO: change this into a database query

files = {}

for root, dirs, filenames in os.walk('data'):
    for f in filenames:

        # get the folder name after data\
        # i.e. emobility_json or hafengebiets_json
        folder = root.split('\\')[1]

        # create a new list if the folder name is not in the dictionary
        if folder not in files:
            files[folder] = []
        
        # append the file name to the list
        files[folder].append(f)

In [4]:
# takes in a path to a file and returns the json object
def load_json(json_path, encoding='utf-8'):
    json_data = {}
    with open(json_path, 'r', encoding=encoding) as settings_file:
        json_data = json.load(settings_file)
    return json_data

## Possible colors for Folium:

```{'lightgray', 'gray', 'beige', 'darkgreen', 'black', 'red', 'darkblue', 'darkred', 'lightblue', 'green', 'white', 'cadetblue', 'pink', 'lightred', 'lightgreen', 'darkpurple', 'purple', 'blue', 'orange'}```

Or just use a hex color code, e.g. ```'#FF0000'```

In [5]:
m = folium.Map(location=(53.55, 9.99), zoom_start=12)

In [6]:
for category, file_list in files.items():
    
    # Load the settings for this category if it exists.
    settings_path = os.path.join('data', category, 'settings.json')
    category_settings = load_json(settings_path) if os.path.exists(settings_path) else {}
    
    # Create a feature group for this category using the display_name from settings.
    feature_group_name = category_settings.get('display_name', category.replace('_json', '').replace('_', ' ').title())
    feature_group = folium.FeatureGroup(name=feature_group_name, show=False)

    marker_cluster = None
    
    # display the feature groups on the map
    for file_name in file_list:

        # Skip the settings.json file.
        if file_name == "settings.json":
            continue
        
        # Only add files with coordinates in EPSG:4326 for compatibility.
        if 'EPSG_4326' in file_name:
            geojson_path = os.path.join('data', category, file_name)
            geojson_data = load_json(geojson_path)
            # print("Loaded", geojson_path)
            
            file_settings = category_settings.get('files', {}).get(file_name, {})
            popup_properties = file_settings.get('popup_properties', {})
            
            for feature in geojson_data['features']:

                # compute the style for a geojson object
                def compute_style_function(file_settings):
                    """ Returns a style function configured with the given file settings. """
                    def compute_style(feature):
                        # Default color from settings
                        fillColor = file_settings.get('color', 'blue')
                        color = file_settings.get('color', 'black')
                        line_weight = file_settings.get('line_weight', 8)
                        
                        # If there's a colormap, compute the color based on feature value
                        if file_settings.get('colormap', False):
                            colormap_data = file_settings.get('colormap')
                            colormap_property = colormap_data["property"]
                            feature_value = feature['properties'][colormap_property]
                            colormap = cm.LinearColormap(colors=colormap_data["colors"], vmin=colormap_data["vmin"], vmax=colormap_data["vmax"])
                            color = colormap(feature_value)[:7]  # Remove alpha channel from color

                        return {
                            'fillColor': fillColor,
                            'color': color,
                            'weight': line_weight
                        }
                    return compute_style

                # populate the popup content with the properties specified in settings
                # The first line is the name of the marker
                marker_name = file_settings.get('name', '')
                popup_content = f'<b>{marker_name}</b><br>'

                # Afterwards, each line is a property of the marker, as specified in settings.json
                popup_content += '<br>'.join(
                    [f"<b>{display_name}</b>: {feature['properties'][internal_name]}" for display_name, internal_name in popup_properties.items() if internal_name in feature['properties']]
                )

                feature_type = file_settings.get('type')
                
                # Add the feature to the map. Feature type is specified in settings.json
                if feature_type == "geojson":

                    style_function = compute_style_function(file_settings)

                    folium.GeoJson(
                        feature,
                        style_function=style_function,
                        tooltip=popup_content
                    ).add_to(feature_group)
                
                elif feature_type == "marker":
                    coords = feature['geometry']['coordinates']

                    # get the icon data from settings.json
                    icon = file_settings.get('icon', ''), 
                    icon_prefix = file_settings.get('icon-prefix', '')
                    icon_color = file_settings.get('color', 'blue')

                    folium.Marker(
                        location=[coords[1], coords[0]],  # Swap coords for latitude and longitude
                        popup=popup_content,
                        icon=folium.Icon(icon=icon, prefix=icon_prefix, color=icon_color)
                    ).add_to(feature_group)
                
                elif feature_type == "marker-cluster":

                    if marker_cluster is None:
                        marker_cluster = MarkerCluster().add_to(feature_group)

                    coords = feature['geometry']['coordinates']

                    # get the icon data from settings.json
                    icon = file_settings.get('icon', ''), 
                    icon_prefix = file_settings.get('icon-prefix', '')
                    icon_color = file_settings.get('color', 'blue')

                    folium.Marker(
                        location=[coords[1], coords[0]],  # Swap coords for latitude and longitude
                        popup=popup_content,
                        icon=folium.Icon(icon=icon, prefix=icon_prefix, color=icon_color)
                    ).add_to(marker_cluster)
                
                elif feature_type == "choropleth":
                    # TODO: Add choropleth logic here.
                    pass

    feature_group.add_to(m)

folium.LayerControl().add_to(m)

In [7]:
# m

In [8]:
# save the map to an html file
m.save('map.html')